# Chatbot- Memory optimization
### Trimming history based on last k messages & Summarized memory
##### Boilerplate code

In [4]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

##### Just to know - Manually trimming history

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = {}

def chat_with_bot(payload):
    input = payload['input']
    session_id = payload['session_id']

    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory()
    
    history = conversation[session_id]

    history.messages = history.messages[-2:]
    
    history.add_user_message(input)
    res = chain.invoke({"input": input, "history": history.messages})
    history.add_ai_message(res)
    print(f"history messages length: {len(history.messages)}")
    
    return res


res = chat_with_bot({"input": "My name is Praveen", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is google?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "Where is it located?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is my name?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

### Trimming history in RunnableWithMessageHistory

In [ ]:
from collections import deque
from langchain_core.messages import BaseMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

store = {}

class WindowedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, k: int = 2):
        self._messages = deque(maxlen=k*2)

    @property
    def messages(self):
        return list(self._messages)

    def add_message(self, message: str) -> None:
        self._messages.append(message)

    def clear(self):
        self._messages.clear()


def get_session_history(session_id):
    if session_id not in store:
         store[session_id] = WindowedChatMessageHistory(k=2)
    return store[session_id] # returns list of messages
    # history = store[session_id]
    # # Keep only last 10 messages
    # if len(history.messages) > 10:
    #     history.messages = history.messages[-10:]
    # return history


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)




res = chain_with_history.invoke(
    {"input": "My name is Praveen."},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "Who is Donald trump?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "Who is Putin?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


### Summarized memory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage


class SummarizedHistory:
    def __init__(self):
        self.summary = "The conversation is empty."

    def update(self, user_msg, ai_msg):
        # Summarize old summary + new turn
        summarize_prompt = ChatPromptTemplate.from_messages([
            ("system", "Summarize the conversation so far briefly."),
            ("human", "Previous summary:\n{summary}\n\nNew exchange:\nUser: {user}\nAI: {ai}\n\nUpdated summary:")
        ])
        chain = summarize_prompt | google_llm | StrOutputParser()
        res = chain.invoke({
            "summary": self.summary,
            "user": user_msg,
            "ai": ai_msg
        })
        self.summary = res



prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all my questions shortly and briefly from now on.'),
    ('system', 'Here is the previous conversation summary for your reference: {summary}'),
    ('human', '{input}')
])

chain = prompt_template | google_llm | StrOutputParser()

# store = {session_id: summary, session_id: summary}
store = {}

# Create / get history per session
def get_session(session_id):
    if session_id not in store:
        store[session_id] = SummarizedHistory()
    return store[session_id]

# Example use
session = get_session("abc123")




In [ ]:
user_input = "Hey, I’m Praveen. I like Python."
ai_output = chain.invoke({"input": user_input, "summary": session.summary})
session.update(user_input, ai_output)
print("AI:", ai_output, "\n", "Summary:", session.summary, "\n")

user_input = "Remind me what i just said"
ai_output = chain.invoke({"input": user_input, "summary": session.summary})
session.update(user_input, ai_output)
print("AI:", ai_output, "\n", "Summary:", session.summary, "\n")
